In [1]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [2]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [3]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [4]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Obtener el nombre de la columna objetivo desde la Serie y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista


In [44]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'

In [ ]:
selected = ['pahosp1y','padoctor1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']
len(selected)

111

In [25]:
selected=f85

In [ ]:
selected = list(set(selected))

df = df.dropna(subset=[target]) 
df = df[selected].copy()

In [ ]:
porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.5)]

In [ ]:
# fill
y = df[target]
X = df.drop(target,axis=1)

In [ ]:
from src import feature_selection as fs
X = fs.fast_fill(X)

In [ ]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [ ]:
# # resample 
# X_train, y_train = dp.apply_resample(X_train, y_train,v=2)

In [ ]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [ ]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [ ]:
params={'max_depth': 28, 'subsample': 0.6559545947005226, 'colsample_bytree': 0.9075616505206274, 'eta': 0.03568753700048324, 'min_child_weight': 2, 'learning_rate': 0.0869908981133205, 'n_estimators': 571}

In [ ]:
model = XGBClassifier(**params)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

AUC=0.8970324396109357
pahosp1y
0.0    20591
1.0     2919
Name: count, dtype: int64


In [ ]:
f50=feature_importance(model, X, y, 50)
f50.append('pahosp1y')
print(f50)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'pahosp1y']


In [ ]:
f55=feature_importance(model, X, y, 55)
f55.append('pahosp1y')
print(f55)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'parafaany', 'pawalkr', 'pamoney', 'parfaany', 'padrinkb', 'pahosp1y']


In [ ]:
f65=feature_importance(model, X, y, 65)
f65.append('pahosp1y')
print(f65)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'parafaany', 'pawalkr', 'pamoney', 'parfaany', 'padrinkb', 'pacage', 'pamhip', 'pabreath_m', 'pabed', 'paoangry', 'pafatigue', 'pameals', 'pamammog', 'parjudg', 'paeat', 'pahosp1y']


In [ ]:
f70=feature_importance(model, X, y, 70)
f70.append('pahosp1y')
print(f70)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'parafaany', 'pawalkr', 'pamoney', 'parfaany', 'padrinkb', 'pacage', 'pamhip', 'pabreath_m', 'pabed', 'paoangry', 'pafatigue', 'pameals', 'pamammog', 'parjudg', 'paeat', 'parxhibp', 'papaina', 'parxdiab', 'paswell', 'parorgnz', 'pahosp1y']


In [ ]:
f75=feature_importance(model, X, y, 75)
f75.append('pahosp1y')
print(f75)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'parafaany', 'pawalkr', 'pamoney', 'parfaany', 'padrinkb', 'pacage', 'pamhip', 'pabreath_m', 'pabed', 'paoangry', 'pafatigue', 'pameals', 'pamammog', 'parjudg', 'paeat', 'parxhibp', 'papaina', 'parxdiab', 'paswell', 'parorgnz', 'pasmoken', 'palgmusaa', 'parifaany', 'pabath', 'pamobilaa', 'pahosp1y']


In [ ]:
raise ValueError('STOP')

In [ ]:
f80=feature_importance(model, X, y, 80)
f80.append('pahosp1y')
print(f80)

['palift', 'padoctor1y', 'paoalchl', 'pashop', 'pacholst', 'pacncrradn', 'pabathehlp', 'pacncrsurg', 'padrinkcr', 'pastroklmt', 'pacncrchem', 'pagender', 'paodngr', 'pafallinj', 'parxhrtat', 'parxstrok', 'palunglmt_m', 'pasit', 'pacancre', 'padresshlp', 'parfcaren', 'parxlung_m', 'paurina2y', 'pahearaid', 'pahrtatlmt', 'paarthlmt', 'pacncrmeds', 'pabedhlp', 'pamedhlp', 'pahigov', 'pararcare', 'pahipe_m', 'paeathlp', 'patoilethlp', 'padrinkbd', 'pahipcomp', 'pamoneyhlp', 'pahrtatte', 'paoplot', 'pameds', 'paarms', 'pastroke', 'paprost', 'pawalkhlp', 'pamealhlp', 'parxarthr', 'parechrtatt', 'pashophlp', 'pagrossaa', 'pacncrothr', 'parafaany', 'pawalkr', 'pamoney', 'parfaany', 'padrinkb', 'pacage', 'pamhip', 'pabreath_m', 'pabed', 'paoangry', 'pafatigue', 'pameals', 'pamammog', 'parjudg', 'paeat', 'parxhibp', 'papaina', 'parxdiab', 'paswell', 'parorgnz', 'pasmoken', 'palgmusaa', 'parifaany', 'pabath', 'pamobilaa', 'padiabe', 'pasmokev', 'papainfr', 'parxdiabo', 'pafall', 'pahosp1y']


In [ ]:
f85=feature_importance(model, X, y, 85)
f85.append('pahosp1y')
print(f85)

In [ ]:
raise ValueError()

ValueError: 

In [ ]:
import optuna
def objective(trial):
    # Definir los hiperparámetros a sintonizar
    params = {
        'booster': 'gbtree',
        'max_depth': trial.suggest_int('max_depth', 7, 45),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'eta': trial.suggest_float('eta', 0.01, 0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 900),
    }
    
    # Entrenar el modelo con los hiperparámetros seleccionados
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    # Predecir las probabilidades en el conjunto de prueba y calcular el AUC
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    
    return auc